<a href="https://colab.research.google.com/github/ilhamhs-blip/data_sains_project/blob/main/Modul_7_Kelompok_Beginner_13_Data_Sains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
pip install ucimlrepo

In [24]:
!pip install dash

In [25]:
!pip install hvplot

In [26]:
pip install panel hvplot holoviews param

In [27]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
auto_mpg = fetch_ucirepo(id=9)
df = auto_mpg.data.original
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   car_name      398 non-null    object 
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model_year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   mpg           398 non-null    float64
dtypes: float64(4), int64(4), object(1)
memory usage: 28.1+ KB


In [28]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print("Missing value tiap kolom:")
print(df.isnull().sum())
median_hp = df['horsepower'].median()

df['origin'] = pd.to_numeric(df['origin'], errors='coerce')
df['origin'] = df['origin'].map({
    1: 'USA',
    2: 'Europe',
    3: 'Japan'
})

df['horsepower'] = df['horsepower'].fillna(median_hp)
print("\nSetelah imputasi:")
print(df.isnull().sum())

Missing value tiap kolom:
car_name        0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
mpg             0
dtype: int64

Setelah imputasi:
car_name        0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
origin          0
mpg             0
dtype: int64


In [29]:
!pip install jupyter_bokeh

In [43]:
import pandas as pd
import panel as pn
import hvplot.pandas
import holoviews as hv
import param

pn.extension()

df['origin'] = df['origin'].astype(str)
df['cylinders'] = df['cylinders'].astype(str)

# Membuat isi dashboard (visualisasi data dan nilai statistik)
class AutoMPGDashboard(param.Parameterized):
    color_map = {
        'USA': '#1f77b4',    # biru
        'Europe': '#ff7f0e', # oranye
        'Japan': '#2ca02c',   # hijau
        'Average': '#9467bd' # ungu untuk rata-rata
    }

    selected_origin = param.ListSelector(default=['All'], objects=['All'] + sorted(df['origin'].unique()))
    selected_cylinders = param.ListSelector(default=['All'], objects=['All'] + sorted(df['cylinders'].unique()))

    def __init__(self, **params):
        super().__init__(**params)
        self.filtered_df = df.copy()

    @param.depends('selected_origin', 'selected_cylinders')
    def update_data(self):
        dff = df.copy()
        if 'All' not in self.selected_origin:
            dff = dff[dff['origin'].isin(self.selected_origin)]
        if 'All' not in self.selected_cylinders:
            dff = dff[dff['cylinders'].isin(self.selected_cylinders)]
        self.filtered_df = dff
        return dff

    #Scatter plot
    @param.depends('selected_origin', 'selected_cylinders')
    def scatter_plot(self):
        dff = self.update_data()
        if not dff.empty:
            colors = [self.color_map[o] for o in sorted(self.color_map.keys()) if o in dff['origin'].unique()]
            return dff.hvplot.scatter(
                x='weight', y='mpg', by='origin',
                hover_cols=['car_name', 'cylinders', 'horsepower'],
                title="Weight vs MPG",
                xlabel='weight (lbs)', ylabel='mpg',
                height=300, width=550,
                color=hv.Cycle(colors), legend=False
            )
        else:
            return hv.Div("No data to display for selected criteria.")

    @param.depends('selected_origin', 'selected_cylinders')
    def bar_plot(self):
        dff = self.update_data()
        if dff.empty:
            return hv.Div("No data to display for selected criteria.")
        bar_data = dff.groupby('cylinders')['mpg'].mean().reset_index()
        max_mpg = bar_data['mpg'].max()
        bar_data['color'] = bar_data['mpg'].apply(
            lambda x: '#ff7f0e' if x == max_mpg else '#1f77b4'
        )

        # Plot batangnya
        bars = bar_data.hvplot.bar(
            x='cylinders',
            y='mpg',
            color='color',
            hover_cols=['cylinders', 'mpg'],
            title="Average MPG by Cylinders",
            xlabel='cylinders',
            ylabel='average mpg',
            height=300,
            width=550,
            legend=False
        )

        return bars

    @param.depends('selected_origin', 'selected_cylinders')
    def line_plot(self):
        dff = self.update_data()
        plots = []

        # Always show overall average trend
        overall_yearly_avg = df.groupby('model_year')['mpg'].mean().reset_index()
        if not overall_yearly_avg.empty:
            plots.append(overall_yearly_avg.hvplot.line(
                x='model_year', y='mpg', label='Overall Average', color=self.color_map.get('Average'), legend='top_right'
            ) * overall_yearly_avg.hvplot.scatter(
                 x='model_year', y='mpg', size=50, color=self.color_map.get('Average'), legend=False
            ))

        # Show trends for selected origins (excluding 'All')
        selected_origins_without_all = [o for o in self.selected_origin if o != 'All']
        if selected_origins_without_all:
            yearly_avg = df[df['origin'].isin(selected_origins_without_all)].groupby(['model_year', 'origin'])['mpg'].mean().reset_index()
            for origin in sorted(selected_origins_without_all):
                 origin_data = yearly_avg[yearly_avg['origin'] == origin]
                 if not origin_data.empty:
                    plots.append(origin_data.hvplot.line(
                        x='model_year', y='mpg', label=origin, color=self.color_map.get(origin), legend='top_right'
                    ) * origin_data.hvplot.scatter(
                        x='model_year', y='mpg', size=50, color=self.color_map.get(origin), legend=False
                    ))


        if plots:
            # Combine all plots
            combined_plot = plots[0]
            for plot in plots[1:]:
                combined_plot *= plot

            return combined_plot.opts(
                title="MPG Trend by Year",
                height=250, width=1120,
                xlabel='model year', ylabel='average mpg',
                legend_position='bottom_right'
            )
        else:
            return hv.Div("No data to display for selected criteria.")


    @param.depends('selected_origin', 'selected_cylinders')
    def stats_view(self):
        dff = self.update_data()
        print("Filtered DataFrame shape:", dff.shape) # Debugging line
        if not dff.empty:
            avg_mpg = f"{dff['mpg'].mean():.1f}"
            highest_mpg = f"{dff['mpg'].max():.1f}"
            most_cylinders = f"{dff['cylinders'].mode().iloc[0]}"
            total_cars = f"{len(dff)}"
            print("Calculated Average MPG:", avg_mpg) # Debugging line
        else:
            avg_mpg = highest_mpg = most_cylinders = "N/A"
            total_cars = "0"
            print("Filtered DataFrame is empty.") # Debugging line


        # HTML untuk 4 stats
        stats_html = f"""
        <div style="display: grid; grid-template-columns: repeat(4, 1fr);gap: 55px; margin: 20px 0px 0 350px;">
            <div style="background: #e3f2fd; padding: 25px; border-radius: 8px; text-align: center; border-left: 4px solid #2196f3;">
                <div style="font-size: 1.2em; color: #1976d2; font-weight: bold;">Average MPG</div>
                <div style="font-size: 2.4em; font-weight: bold; color: #1976d2;">{avg_mpg}</div>
            </div>
            <div style="background: #e8f5e9; padding: 25px; border-radius: 8px; text-align: center; border-left: 4px solid #4caf50;">
                <div style="font-size: 1.2em; color: #2e7d32; font-weight: bold;">Highest MPG</div>
                <div style="font-size: 2.4em; font-weight: bold; color: #2e7d32;">{highest_mpg}</div>
            </div>
            <div style="background: #fff3e0; padding: 25px; border-radius: 8px; text-align: center; border-left: 4px solid #ff9800;">
                <div style="font-size: 1.2em; color: #ef6c00; font-weight: bold;">Most Common Cylinders</div>
                <div style="font-size: 2.4em; font-weight: bold; color: #ef6c00;">{most_cylinders}</div>
            </div>
            <div style="background: #fce4ec; padding: 25px; border-radius: 8px; text-align: center; border-left: 4px solid #e91e63;">
                <div style="font-size: 1.2em; color: #c2185b; font-weight: bold;">Total Cars</div>
                <div style="font-size: 2.4em; font-weight: bold; color: #c2185b;">{total_cars}</div>
            </div>
        </div>
        """
        return pn.pane.HTML(stats_html)

    @param.depends('selected_origin', 'selected_cylinders')
    def data_table(self):
        dff = self.update_data()
        return pn.widgets.Tabulator(dff, pagination='local', page_size=10, height=300)


# Mennyusun Dashboard dan Widgets
dashboard_obj = AutoMPGDashboard()
origin_selector = pn.widgets.MultiChoice(
    name='Select Origin',
    options=['All'] + sorted(df['origin'].unique()),
    value=['All'],
    width=300
)

cylinder_selector = pn.widgets.MultiChoice(
    name='Select Cylinders',
    options=['All'] + sorted(df['cylinders'].unique()),
    value=['All'],
    width=300
)

# Remove the checkbox for the overall average trend
# show_average_checkbox = pn.widgets.Checkbox(name=dashboard_obj.param.show_average_trend.label, value=dashboard_obj.show_average_trend)


origin_selector.link(dashboard_obj, value='selected_origin')
cylinder_selector.link(dashboard_obj, value='selected_cylinders')
# Remove the link for the checkbox
# show_average_checkbox.link(dashboard_obj, value='show_average_trend')


# Header Dashboard
header_html = """
<div style="text-align: center; color: white; background: linear-gradient(45deg, #007bff, #0056b3);
            padding: 20px; border-radius: 10px; margin-bottom: 20px; width: 1500px">
    <h1 style="margin: 0; font-size: 3.0em;">🚗 AUTO MPG DASHBOARD 🚗</h1>
    <p style="margin: 10px 0 0 0; opacity: 0.9;">Explore car specifications and fuel efficiency</p>
</div>
"""

# Info panel
info_panel = pn.Card(
    pn.pane.Markdown("""
    ### 📊 Dashboard Info

    Explore relationships between car specifications and fuel efficiency.

    **Metrics:**
    - **MPG**: Miles per Gallon (fuel efficiency)
    - **Weight**: Vehicle weight in pounds
    - **Horsepower**: Engine power
    - **Cylinders**: Number of engine cylinders
    - **Origin**: Manufacturing region

    Selecting "All" in the **Select Origin** filter shows the overall data and trends across all origins. To see the trend for specific origins, select only those origins from the list.
    """),
    title="ℹ️ Information",
    collapsible=False,
    styles={'background': '#f8f9fa', 'max-width' : '320px'}
)

# Control Panel (Filtering)
control_panel = pn.Card(
    pn.Column(
        pn.pane.Markdown("### 🔧 Filters"),
        origin_selector,
        cylinder_selector,
        # Remove the checkbox from the control panel
        # show_average_checkbox,
        info_panel
    ),
    title="⚙️ Control Panel",
    collapsible=False,
    styles={
        'background': '#f8f9fa',
        'margin-top': '-120px',
        'max-width': '320px'
    }
)

# Struktur DashboardDashboard
dashboard = pn.Column(
    pn.pane.HTML(header_html),
    dashboard_obj.stats_view,
    pn.Row(
        control_panel,
        pn.Column(
            pn.Row(
                dashboard_obj.scatter_plot,
                dashboard_obj.bar_plot
            ),
            dashboard_obj.line_plot
        )
    ),
    pn.pane.Markdown("## 📋 Car Data Table"),
    dashboard_obj.data_table,
    sizing_mode='fixed'
)

dashboard

Filtered DataFrame shape: (398, 9)
Calculated Average MPG: 23.5


Column(sizing_mode='fixed')
    [0] HTML(str)
    [1] ParamMethod(method, _pane=HTML, defer_load=False)
    [2] Row
        [0] Card(collapsible=False, styles={'background': '#f8f9fa', ...}, title='⚙️ Control Panel')
            [0] Column
                [0] Markdown(str)
                [1] MultiChoice(name='Select Origin', options=['All', 'Europe', ...], value=['All'])
                [2] MultiChoice(name='Select Cylinders', options=['All', '3', '4', ...], value=['All'])
                [3] Card(collapsible=False, styles={'background': '#f8f9fa', ...}, title='ℹ️ Information')
                    [0] Markdown(str)
        [1] Column
            [0] Row
                [0] ParamMethod(method, _pane=HoloViews, defer_load=False)
                [1] ParamMethod(method, _pane=HoloViews, defer_load=False)
            [1] ParamMethod(method, _pane=HoloViews, defer_load=False)
    [3] Markdown(str)
    [4] ParamMethod(method, _pane=Tabulator, defer_load=False)